# Notebook for retrieving, processing, and saving daily (aggregated) UK rainfall data via the EA API

In [4]:
# importing required libraries
import numpy as np
import pandas as pd
import json
import requests
from datetime import datetime, date
import os

In [36]:
# printing current date and time
print('Date and time: ' + str(datetime.today().replace(microsecond=0)))

Date and time: 2023-05-21 16:09:36


In [6]:
# list present working directory
#os.getcwd()

## Retrieving the data

In [23]:
# data retrieval functions - only for stations and measures via the API

# function to retrieve data on rainfall stations
def get_data_stations(parameter_name = None,
                 parameter = None,
                 qualifier = None,
                 label = None,
                 town = None,
                 catchment_name = None,
                 river_name = None,
                 station_reference = None,
                 rloi_id = None,
                 search = None,
                 lat = None,
                 long = None,
                 d = None,
                 type = None,
                 status = None):
    """Get details of rainfall monitoring stations from the EA API
    
      Query parameter details:
    
      :param parameter_name: Return only those stations which measure parameters with the given name, for example Rainfall, Water Level or Flow.
      :param parameter: Return only those stations which measure parameters with the given short form name, for example rainfall, level or flow.
      :param qualifier: Return only those stations which measure parameters with qualifier. Useful qualifiers are Stage and Downstream Stage (for stations such as weirs which measure levels at two locations), Groundwater for groundwater levels as opposed to river levels and Tidal Level for tidal levels.
      :param label: Return only those stations whose label is exactly as given.
      :param town: Return only those stations whose town is as given. Not all stations have an associated town.
      :param catchment_name: Return only those stations whose catchment name is exactly as given. Not all stations have an associated catchment area.
      :param river_name: Return only those stations whose river name is exactly as given. Not all stations have an associated river name.
      :param station_reference: Return only those stations whose reference identifier is as given. The station reference is an internal identifier used by the Environment Agency.
      :param rloi_id: Return only the station (if there is one) whose RLOIid (River Levels on the Internet identifier) matches.
      :param search: Return only those stations whose label contains the given value.
      :param lat: Return those stations whose location falls within d km of the given latitude/longitude (in WGS84 coordinates), this may be approximated by a bounding box.
      :param long: Return those stations whose location falls within d km of the given latitude/longitude (in WGS84 coordinates), this may be approximated by a bounding box.
      :param d: Return those stations whose location falls within d km of the given latitude/longitude (in WGS84 coordinates), this may be approximated by a bounding box.
      :param type: Return only those stations of the given type, where type can be one of "SingleLevel", "MultiTraceLevel", "Coastal", "Groundwater" or "Meteorological"
      :param status: Return only those stations with the given status. Can be one of "Active", "Closed" or "Suspended".
      """
    
    # URL of the UK EA API
    api_url = "https://environment.data.gov.uk/flood-monitoring/id/stations"
 
    # build a dictionary of the query parameters
    params =  {'parameterName': parameter_name,
                'parameter': parameter,
                'qualifier': qualifier,
                'label': label,
                'town': town,
                'catchmentName': catchment_name,
                'riverName': river_name,
                'stationReference': station_reference,
                'RLOIid': rloi_id,
                'search': search,
                'lat': lat,
                'long': long,
                'dist': d,
                'type': type,
                'status': status
              }
    
    # Get data on stations from the EA API
    response = requests.get(api_url, 
                            params = params)

    # ensuring that cookies are allowed
    #response = requests.get(api_url, params, cookies=response.cookies)
    
    # Extracting JSON data from the response container
    data = response.json()

    # Loading data to a Pandas data frame
    stations = pd.DataFrame(data["items"])

    return(stations)


# function to retrieve data on rainfall measures
def get_data_measures(parameter_name = None,
                     parameter = None,
                     qualifier = None,
                     station_reference = None,
                     station = None,
                     search = None):
    """Get details of measures available from the EA API

       :param parameter_name: Return only measures for parameters with the given name, for example Water Level or Flow.
       :param parameter: Return only measures for parameters with the given short form name, for example level or flow.
       :param qualifier: Return only those measures with qualifier. Useful qualifiers are Stage and Downstream Stage (for stations such as weirs which measure levels at two locations), Groundwater for groundwater levels as opposed to river levels and Tidal Level for tidal levels.
       :param station_reference: Return only those measures which are available from the station with the given reference identifier.
       :param station: Return only those measures which are available from the station with the given URI.
       :param search: Return only those measures whose label contains the given value.
    """
    api_url = "http://environment.data.gov.uk/flood-monitoring/id/measures"
    
     # build a dictionary of the query parameters
    params = {'parameterName': parameter_name,
    'parameter': parameter,
    'qualifier': qualifier,
    'stationReference': station_reference,
    'station': station,
    'search': search
    }
    
    # Get data about measures from the EA API
    response = requests.get(api_url, 
                            params)

    # ensuring that cookies are allowed
    #response = requests.get(api_url, params, cookies=response.cookies)
    
    # Extract JSON data from the response
    data = response.json()

    # Load data to a data frame
    measures = pd.DataFrame(data["items"])

    return(measures)

In [24]:
# retrieving the rainfall stations and measures data
rainfall_stations_data = get_data_stations(parameter='rainfall')
rainfall_measures_data = get_data_measures(parameter='rainfall')

# retrieving the rainfall readings archive for today
rainfall_readings_today = pd.read_csv('http://environment.data.gov.uk/flood-monitoring/data/readings.csv?parameter=rainfall&_view=full&date=' + str(date.today()))

In [25]:
# Checking the number of entries for each dataset
print(rainfall_stations_data.shape,
      rainfall_measures_data.shape,
      rainfall_readings_today.shape)

(996, 19) (1023, 13) (40995, 13)


## Data cleaning and processing

In [41]:
# aggregating data by date for all station IDs
rainfall_readings_aggregated_values = rainfall_readings_today.groupby(['stationReference'], as_index=False)['value'].sum()

In [42]:
# merging the 3 dataframes for rainfall stations, measures, and daily reaadings

# left join of readings with measures to keep all readings and outer join of resulting df with stations to keep
# details of non-responsive stations as well
rainfall_readings_aggregated = rainfall_readings_aggregated_values.merge(rainfall_measures_data,on='stationReference', how='left').merge(rainfall_stations_data,on='stationReference', how='outer')

# viewing the shape of the resulting df
print(rainfall_readings_aggregated.shape)

(1028, 32)


In [45]:
rainfall_readings_aggregated.iloc[0,:]

date                                                     2023-05-21
time                                                       16:14:59
station_id                                                   000008
station_type                                       Rainfall station
station_grid_reference                                     SK371983
station_latitude                                          53.480556
station_longitude                                         -1.441674
station_easting                                            437150.0
station_northing                                           398350.0
parameter_id                                               rainfall
parameter_name                                             Rainfall
reading_qualifier                          Tipping Bucket Raingauge
reading_value                                                   0.0
reading_unit_id           http://qudt.org/1.1/vocab/unit#Millimeter
reading_unit_name                               

In [44]:
# further data cleaning

# dropping duplicate and redundant columns
rainfall_readings_aggregated = rainfall_readings_aggregated.drop(columns=['@id_x',
                                                              'label_x',
                                                              'measures',
                                                              'latestReading',
                                                              'notation_x',
                                                              'station',
                                                              '@id_y',
                                                              'notation_y',
                                                              # columns relevant only for river data
                                                              'catchmentName',
                                                              'dateOpened',
                                                              'riverName',
                                                              'stageScale',
                                                              'status',
                                                              'town',
                                                              'wiskiID',
                                                              'datumOffset',
                                                              'RLOIid'
                                                             ])

# adding date and time columns

# creating date variable
rainfall_readings_aggregated['date'] = str(date.today())

# creaating time variable
rainfall_readings_aggregated['time'] = str(datetime.now().time().replace(microsecond=0))

# renaming columns
rainfall_readings_aggregated.rename(columns = {
                                           'value': 'reading_value',
                                           'parameter': 'parameter_id',
                                           'parameterName': 'parameter_name',
                                           'period': 'reading_period',
                                           'qualifier': 'reading_qualifier',
                                           'stationReference': 'station_id',
                                           'unit':'reading_unit_id',
                                           'unitName':'reading_unit_name',
                                           'valueType':'reading_value_type',
                                           'easting':'station_easting',
                                           'northing': 'station_northing',
                                           'gridReference': 'station_grid_reference',
                                           'label_y': 'station_type',
                                           'lat': 'station_latitude',
                                           'long':'station_longitude',
                                           'valueType':'reading_value_type',
                                           'gridReference': 'station_grid_reference'},
                              inplace = True)
                                                           
# reordering columns
rainfall_readings_aggregated = rainfall_readings_aggregated[['date',
                                                    'time',
                                                    'station_id',
                                                    'station_type',
                                                    'station_grid_reference',
                                                    'station_latitude',
                                                    'station_longitude',
                                                    'station_easting',
                                                    'station_northing',
                                                    'parameter_id',
                                                    'parameter_name',
                                                    'reading_qualifier',
                                                    'reading_value',
                                                    'reading_unit_id',
                                                    'reading_unit_name',
                                                    'reading_value_type',
                                                    'reading_period'
                                               ]]

# viewing the shape of the resulting df
print(rainfall_readings_aggregated.shape)

(1028, 17)


## Explanation of columns

__date__: date  
__time__: time  
__station_id__: station identifier  
__station_type__: type of station  
__station_grid_reference__: grid reference for the station, rounded to a 100m grid  
__station_latitude__: latitude coordinates of station  
__station_longitude__: longitude coordinates of station  
__station_easting__: easting coordinates of station  
__station_northing__: northing coordinates of station  
__parameter_id__: short name/id of the quantity being measured  
__parameter_name__: name of the quantity being measured  
__reading_qualifier__: a qualifier for the quantity being measured, "Tipping Bucket Raingauge" for rainfall  
__reading_value__: the value of the reading for the associated measurement  
__reading_unit_id__: unit id/url for the reading  
__reading_unit_name__: unit name for the reading  
__reading_value_type__: type of measurement, e.g., total, mean, etc.  
__reading_period__: the period between successive readings, in seconds

# Data quality checks

In [46]:
# check if there are any stations without recent rainfall measurements and list their Station IDs

# list total stations with no recent rainfall measurements
print( "Total stations with no recent rainfall measurements: " + str(rainfall_readings_aggregated['reading_value'].isna().sum()))

print('')

print('IDs of the respective stations:')

print('')

# listing their IDs
print(rainfall_readings_aggregated[rainfall_readings_aggregated['reading_value'].isna() == True]['station_id'].values)

Total stations with no recent rainfall measurements: 95

IDs of the respective stations:

['4163' 'E1310' '0' 'E1179' '1117' '45101' '270400TP' '055223' 'E15520'
 'E1965' 'E11060' '008632' '000032' 'E23633' 'E24947' '1760' '023839'
 'E23535' '019356' 'E43041' '266474TP' '577805' 'E42971' 'E22876' 'E11040'
 '000102TP' 'E22231' '49106' '592848' '4103' '550148' 'E21637' '586493'
 '1795' 'E23657' '073422' '50108' '1409' 'E7120' 'E22735' '238097TP'
 '593321' 'E2450' '562992' 'E1691' '45157' '007533' '584098' 'E14880'
 '46101' 'E2859' '476898_TG_333' '246847TP' 'E24775' '021028' 'E6780'
 '568363' 'E1928' '585022' 'E24585' '605382' 'E2433' '563599' '55000A'
 'E24141' '562811' 'E24499' 'E5720' '45108_' 'E1711' '4527' '6666LO'
 '1409Lucas' '4549' '4548' 'E5650' '7015' '1607' '47176' '1338' '1750'
 '1810' '1408' 'Coldw1' 'Mitch1' 'Edgeh1' 'Dubbs1' '585122_' '2102SO'
 '575092______________________________________'
 '574786______________________________________' 'E84360' '062916' 'E60880'
 'NE063'

In [47]:
# NaN value check with an assert statement
#assert len(rainfall_readings_aggregated[rainfall_readings_aggregated['reading_value'].isna() == True]['station_id'].tolist()) == 0, f"0 expected, got: {len(rainfall_readings_aggregated[rainfall_readings_aggregated['reading_value'].isna() == True]['station_id'].tolist())}"

AssertionError: 0 expected, got: 95

In [49]:
# listing stations without latitude (can also be checked for longitude)
print( "Total stations without latitude coordinates: " + str(rainfall_readings_aggregated['station_latitude'].isna().sum()))

print('')

print('IDs of the respective stations:')

print('')

# listing their IDs
print(rainfall_readings_aggregated[rainfall_readings_aggregated['station_latitude'].isna() == True]['station_id'].values)

Total stations without latitude coordinates: 10

IDs of the respective stations:

['024121' '068416' '282947TP' 'E23500' 'E7040' 'Not_Specified'
 'Not_Specified' '6666LO' '4549' '4548']


In [50]:
# check for duplicate stations by station ID
rainfall_readings_aggregated['station_id'].is_unique

False

In [51]:
rainfall_readings_aggregated[rainfall_readings_aggregated.duplicated(['station_id'])]

,date,time,station_id,station_type,station_grid_reference,station_latitude,station_longitude,station_easting,station_northing,parameter_id,parameter_name,reading_qualifier,reading_value,reading_unit_id,reading_unit_name,reading_value_type,reading_period
134,2023-05-21,16:14:59,1112,Rainfall station,ST888984,51.684691,-2.162673,388850.0,198450.0,rainfall,Rainfall,Tipping Bucket Raingauge,0.0,http://qudt.org/1.1/vocab/unit#Millimeter,mm,event,NaN
139,2023-05-21,16:14:59,1140,Rainfall station,SO987196,51.875411,-2.019556,398750.0,219650.0,rainfall,Rainfall,Tipping Bucket Raingauge,0.0,http://qudt.org/1.1/vocab/unit#Millimeter,mm,event,NaN
157,2023-05-21,16:14:59,1606,Rainfall station,SO495284,51.952233,-2.735485,349550.0,228450.0,rainfall,Rainfall,Logged,0.0,http://qudt.org/1.1/vocab/unit#Millimeter,mm,event,NaN
228,2023-05-21,16:14:59,261602TP,Rainfall station,SP905090,51.772802,-0.689041,490550.0,209050.0,rainfall,Rainfall,Water,0.0,http://qudt.org/1.1/vocab/unit#Millimeter,mm,total,900.0
284,2023-05-21,16:14:59,305111,Rainfall station,TR210379,51.098221,1.155768,621050.0,137950.0,rainfall,Rainfall,Tipping Bucket Raingauge,0.0,http://qudt.org/1.1/vocab/unit#Millimeter,mm,total,900.0
326,2023-05-21,16:14:59,3629,Rainfall station,SK609215,52.788080,-1.097664,460950.0,321550.0,rainfall,Rainfall,Tipping Bucket Raingauge,0.0,http://qudt.org/1.1/vocab/unit#Millimeter,mm,event,NaN
523,2023-05-21,16:14:59,551440,Rainfall station,SJ431758,53.276556,-2.854006,343150.0,375850.0,rainfall,Rainfall,Tipping Bucket Raingauge,0.0,NaN,---,total,900.0
527,2023-05-21,16:14:59,553564,Rainfall station,SJ665574,53.113166,-2.501173,366550.0,357450.0,rainfall,Rainfall,Tipping Bucket Raingauge,0.0,NaN,---,total,900.0
534,2023-05-21,16:14:59,559100R,Rainfall station,SK044958,53.459370,-1.934440,404450.0,395850.0,rainfall,Rainfall,Tipping Bucket Raingauge,0.0,NaN,---,total,900.0
544,2023-05-21,16:14:59,562656,Rainfall station,SD768048,53.539768,-2.350780,376850.0,404850.0,rainfall,Rainfall,Tipping Bucket Raingauge,0.0,NaN,---,total,900.0


In [ ]:
# checking if the coordinates of all rainfall stations are in the UK

#lat = 61.5 and 49
#long = -11 to 3

# Saving processed data

In [52]:
# saving combined dataset as csv and excel files
rainfall_readings_aggregated.to_csv(f'../data/rainfall_data_aggregated_{date.today()}.csv')
rainfall_readings_aggregated.to_excel(f'../data/rainfall_data_aggregated_{date.today()}.xlsx')